# Title
[]()

In [41]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
import json


In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
from wrangling import filter_df_all_conditions, filter_df_any_condition

# Set up

In [1]:
import json

with open("../online-PT-social-media-NLP/notebooks/credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['am_ig_user_id']
# access_token = credentials['am_ig_access_token']
# group_id = credentials['STEMM_group_id']
access_token = credentials['am_fb_access_token']
group_id = credentials['SSC_group_id']

In [2]:
posts_data_dict = dict()
posts_df_dict = dict()
comments_response_dict = dict()
comments_df_dict = dict()
replies_dict = dict()


# Functions from [previous note book](../online-PT-social-media-NLP/notebooks/2023-03-22%20ChatGPT%20GUI%20FB%20group.ipynb')

In [ ]:
## I will use this as part of the final code for the task
import requests
import pandas as pd

def get_facebook_posts(group_id, access_token, limit=25):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        pandas.DataFrame: A DataFrame containing the posts and their associated data.
    """
    url = f"https://graph.facebook.com/{group_id}/feed"

    params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(10).summary(true),reactions.limit(500).summary(true).type(LIKE).as(like),reactions.limit(500).summary(true).type(LOVE).as(love),reactions.limit(500).summary(true).type(WOW).as(wow),reactions.limit(500).summary(true).type(HAHA).as(haha),reactions.limit(500).summary(true).type(SAD).as(sad),reactions.limit(500).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    print("Response Status Code:", status_code)
    data = response.json()
    return data

## I will use this as part of the final code for the task
def process_response(data):
    posts = []
    comments = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })

    df = pd.DataFrame(posts)
    # Reorder columns
    df = df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow_count", "haha_count", "sad_count", "angry_count", "permalink_url"]]
    
    return df

def get_fb_comments(group_id, access_token, limit=25):
    """
    Pulls comments from a Facebook group using the Facebook API and outputs the results into a pandas DataFrame.
    
    Parameters:
    group_id (str): The ID of the Facebook group to pull comments from.
    access_token (str): A Facebook access token with the necessary permissions to access the group.
    
    Returns:
    Tuple(pandas.DataFrame, dict): A tuple containing a DataFrame with each comment as a row along with its details and replies,
    and the full JSON response from the API.
    """
    # Set up the API endpoint and parameters
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "fields": "comments{created_time,message,from,comments{created_time,message,from}}",
        "access_token": access_token,
        "limit": limit
    }

    # Make the GET request
    response = requests.get(url, params=params)
    
    # Convert the response to a DataFrame
    comments = []
    for post in response.json()["data"]:
        post_comments = post.get("comments", {}).get("data", [])
        for comment in post_comments:
            comment_dict = {
                "post_id": post.get("id"),
                "created_time": comment.get("created_time"),
                "message": comment.get("message"),
                "author_id": comment.get("from", {}).get("id"),
                "author_name": comment.get("from", {}).get("name"),
                "parent_id": None,
                "parent_name": None,
            }
            parent_comment = comment.get("comments", {}).get("data", [])
            if parent_comment:
                parent_comment = parent_comment[0]
                comment_dict["parent_id"] = parent_comment.get("id")
                comment_dict["parent_name"] = parent_comment.get("from", {}).get("name")
            comments.append(comment_dict)
    df = pd.DataFrame(comments)
    return df, response.json()

def create_reply_dataframe(json_object):
    # Initialize empty lists to store data
    post_id_list = []
    parent_comment_message_list = []
    comment_id_list = []
    reply_comment_message_list = []
    reply_created_time_list = []
    reply_id_list = []
    
    # Loop through each post in the json object
    for post in json_object:
        # Get the post ID
        post_id = post['id']
        
        try: # SH 2023-03-22 10:50 Added this 
            # Loop through each comment in the post
            for comment in post['comments']['data']:
                # Get the comment ID and message
                comment_id = comment['id']
                parent_comment_message = comment['message']
                
                # Check if the comment has any replies
                # if 'comments' in comment:
                if 'comments' in comment:
                    # Loop through each reply to the comment
                    for reply in comment['comments']['data']:
                        # Get the reply ID, message, and creation time
                        reply_id = reply['id']
                        reply_comment_message = reply['message']
                        reply_created_time = reply['created_time']
                        
                        # Add the data to the lists
                        post_id_list.append(post_id)
                        parent_comment_message_list.append(parent_comment_message)
                        comment_id_list.append(comment_id)
                        reply_comment_message_list.append(reply_comment_message)
                        reply_created_time_list.append(reply_created_time)
                        reply_id_list.append(reply_id)
        except:
            pass
    
    # Create a dataframe from the lists
    df = pd.DataFrame({
        'post_id': post_id_list,
        'parent_comment_message': parent_comment_message_list,
        'comment_id': comment_id_list,
        'reply_comment_message': reply_comment_message_list,
        'reply_created_time': reply_created_time_list,
        'reply_id': reply_id_list
    })
    return df

def merge_dataframes(comments_df, posts_df, replies_df):
    # merge comments and posts dataframes on post_id
    merged_df = pd.merge(comments_df, posts_df, on='post_id', how='outer')
    # merge replies dataframe on comment_id
    merged_df = pd.merge(merged_df, replies_df, left_on='message_x', right_on='parent_comment_message', how='left')
    # rename columns
    merged_df = merged_df.rename(columns={
        'post_id_x': 'post_id',
        'created_time_x': 'comment_created_time',
        'created_time_y': 'post_created_time',
        'message_x': 'comment_message',
        'message_y': 'post_message',
        'parent_name_x': 'parent_name',
        'reply_comment_message': 'reply_message',
        'reply_created_time': 'reply_created_time',
        'reply_id': 'reply_id'
    })
    # drop unnecessary columns
    merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
    # reorder columns
    merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                           'comment_id', 'reply_message', 'reply_created_time', 'reply_id']]
    return merged_df


# Get Facebook group posts
posts_data = get_facebook_posts(group_id, access_token)
posts_df = process_response(posts_data)

comments_df, comments_response = get_fb_comments(group_id, access_token)
replies = create_reply_dataframe(comments_response['data'])
merged = merge_dataframes(comments_df, posts_df, replies)

# Iteration 1: Initial modification

In [4]:

import requests
import pandas as pd

def get_facebook_posts(group_id, access_token, limit=10):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        pandas.DataFrame: A DataFrame containing the posts and their associated data.
    """
    url = f"https://graph.facebook.com/{group_id}/feed"

    params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(10).summary(true),reactions.limit(500).summary(true).type(LIKE).as(like),reactions.limit(500).summary(true).type(LOVE).as(love),reactions.limit(500).summary(true).type(WOW).as(wow),reactions.limit(500).summary(true).type(HAHA).as(haha),reactions.limit(500).summary(true).type(SAD).as(sad),reactions.limit(500).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    print("Response Status Code:", status_code)
    data = response.json()
    return data

## I will use this as part of the final code for the task
def process_response(data):
    posts = []
    comments = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })

    df = pd.DataFrame(posts)
    # Reorder columns
    df = df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow_count", "haha_count", "sad_count", "angry_count", "permalink_url"]]
    
    return df

def get_fb_comments(group_id, access_token, limit=10):
    """
    Pulls comments from a Facebook group using the Facebook API and outputs the results into a pandas DataFrame.
    
    Parameters:
    group_id (str): The ID of the Facebook group to pull comments from.
    access_token (str): A Facebook access token with the necessary permissions to access the group.
    
    Returns:
    Tuple(pandas.DataFrame, dict): A tuple containing a DataFrame with each comment as a row along with its details and replies,
    and the full JSON response from the API.
    """
    # Set up the API endpoint and parameters
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "fields": "comments{created_time,message,from,comments{created_time,message,from}}",
        "access_token": access_token,
        "limit": limit
    }

    # Make the GET request
    response = requests.get(url, params=params)
    
    # Convert the response to a DataFrame
    comments = []
    for post in response.json()["data"]:
        post_comments = post.get("comments", {}).get("data", [])
        for comment in post_comments:
            comment_dict = {
                "post_id": post.get("id"),
                "created_time": comment.get("created_time"),
                "message": comment.get("message"),
                "author_id": comment.get("from", {}).get("id"),
                "author_name": comment.get("from", {}).get("name"),
                "parent_id": None,
                "parent_name": None,
            }
            parent_comment = comment.get("comments", {}).get("data", [])
            if parent_comment:
                parent_comment = parent_comment[0]
                comment_dict["parent_id"] = parent_comment.get("id")
                comment_dict["parent_name"] = parent_comment.get("from", {}).get("name")
            comments.append(comment_dict)
    df = pd.DataFrame(comments)
    return df, response.json()

def create_reply_dataframe(json_object):
    # Initialize empty lists to store data
    post_id_list = []
    parent_comment_message_list = []
    comment_id_list = []
    reply_comment_message_list = []
    reply_created_time_list = []
    reply_id_list = []
    
    # Loop through each post in the json object
    for post in json_object:
        # Get the post ID
        post_id = post['id']
        
        try: # SH 2023-03-22 10:50 Added this 
            # Loop through each comment in the post
            for comment in post['comments']['data']:
                # Get the comment ID and message
                comment_id = comment['id']
                parent_comment_message = comment['message']
                
                # Check if the comment has any replies
                # if 'comments' in comment:
                if 'comments' in comment:
                    # Loop through each reply to the comment
                    for reply in comment['comments']['data']:
                        # Get the reply ID, message, and creation time
                        reply_id = reply['id']
                        reply_comment_message = reply['message']
                        reply_created_time = reply['created_time']
                        
                        # Add the data to the lists
                        post_id_list.append(post_id)
                        parent_comment_message_list.append(parent_comment_message)
                        comment_id_list.append(comment_id)
                        reply_comment_message_list.append(reply_comment_message)
                        reply_created_time_list.append(reply_created_time)
                        reply_id_list.append(reply_id)
        except:
            pass
    
    # Create a dataframe from the lists
    df = pd.DataFrame({
        'post_id': post_id_list,
        'parent_comment_message': parent_comment_message_list,
        'comment_id': comment_id_list,
        'reply_comment_message': reply_comment_message_list,
        'reply_created_time': reply_created_time_list,
        'reply_id': reply_id_list
    })
    return df

def merge_dataframes(comments_df, posts_df, replies_df):
    # merge comments and posts dataframes on post_id
    merged_df = pd.merge(comments_df, posts_df, on='post_id', how='outer')
    # merge replies dataframe on comment_id
    merged_df = pd.merge(merged_df, replies_df, left_on='message_x', right_on='parent_comment_message', how='left')
    # rename columns
    merged_df = merged_df.rename(columns={
        'post_id_x': 'post_id',
        'created_time_x': 'comment_created_time',
        'created_time_y': 'post_created_time',
        'message_x': 'comment_message',
        'message_y': 'post_message',
        'parent_name_x': 'parent_name',
        'reply_comment_message': 'reply_message',
        'reply_created_time': 'reply_created_time',
        'reply_id': 'reply_id'
    })
    # drop unnecessary columns
    merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
    # reorder columns
    merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                           'comment_id', 'reply_message', 'reply_created_time', 'reply_id']]
    return merged_df

iteration = 1

# Get Facebook group posts
posts_data_dict[iteration] = get_facebook_posts(group_id, access_token)

Response Status Code: 200


In [ ]:

posts_df_dict[iteration] = process_response(posts_data_dict[iteration])
posts_df_dict[iteration]

## 1.1

In [10]:

import requests
import pandas as pd

def get_facebook_posts(group_id, access_token, limit=10):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        pandas.DataFrame: A DataFrame containing the posts and their associated data.
    """
    url = f"https://graph.facebook.com/{group_id}/feed"

    params = {
        "access_token": access_token,
        "fields": "from,created_time,message,permalink_url,comments.limit(10).summary(true),reactions.limit(500).summary(true).type(LIKE).as(like),reactions.limit(500).summary(true).type(LOVE).as(love),reactions.limit(500).summary(true).type(WOW).as(wow),reactions.limit(500).summary(true).type(HAHA).as(haha),reactions.limit(500).summary(true).type(SAD).as(sad),reactions.limit(500).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    print("Response Status Code:", status_code)
    data = response.json()
    return data

## I will use this as part of the final code for the task
def process_response(data):
    posts = []
    comments = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })

    df = pd.DataFrame(posts)
    # Reorder columns
    df = df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow_count", "haha_count", "sad_count", "angry_count", "permalink_url"]]
    
    return df

def get_fb_comments(group_id, access_token, limit=10):
    """
    Pulls comments from a Facebook group using the Facebook API and outputs the results into a pandas DataFrame.
    
    Parameters:
    group_id (str): The ID of the Facebook group to pull comments from.
    access_token (str): A Facebook access token with the necessary permissions to access the group.
    
    Returns:
    Tuple(pandas.DataFrame, dict): A tuple containing a DataFrame with each comment as a row along with its details and replies,
    and the full JSON response from the API.
    """
    # Set up the API endpoint and parameters
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "fields": "comments{created_time,message,from{id,name},comments{created_time,message,from{id,name}}}",
        "access_token": access_token,
        "limit": limit
    }

    # Make the GET request
    response = requests.get(url, params=params)
    
    # Convert the response to a DataFrame
    comments = []
    for post in response.json()["data"]:
        post_comments = post.get("comments", {}).get("data", [])
        for comment in post_comments:
            comment_dict = {
                "post_id": post.get("id"),
                "created_time": comment.get("created_time"),
                "message": comment.get("message"),
                "author_id": comment.get("from", {}).get("id"),
                "author_name": comment.get("from", {}).get("name"),
                "parent_id": None,
                "parent_name": None,
            }
            parent_comment = comment.get("comments", {}).get("data", [])
            if parent_comment:
                parent_comment = parent_comment[0]
                comment_dict["parent_id"] = parent_comment.get("id")
                comment_dict["parent_name"] = parent_comment.get("from", {}).get("name")
            comments.append(comment_dict)
    df = pd.DataFrame(comments)
    return df, response.json()

def create_reply_dataframe(json_object):
    # Initialize empty lists to store data
    post_id_list = []
    parent_comment_message_list = []
    comment_id_list = []
    reply_comment_message_list = []
    reply_created_time_list = []
    reply_id_list = []
    
    # Loop through each post in the json object
    for post in json_object:
        # Get the post ID
        post_id = post['id']
        
        try: # SH 2023-03-22 10:50 Added this 
            # Loop through each comment in the post
            for comment in post['comments']['data']:
                # Get the comment ID and message
                comment_id = comment['id']
                parent_comment_message = comment['message']
                
                # Check if the comment has any replies
                # if 'comments' in comment:
                if 'comments' in comment:
                    # Loop through each reply to the comment
                    for reply in comment['comments']['data']:
                        # Get the reply ID, message, and creation time
                        reply_id = reply['id']
                        reply_comment_message = reply['message']
                        reply_created_time = reply['created_time']
                        
                        # Add the data to the lists
                        post_id_list.append(post_id)
                        parent_comment_message_list.append(parent_comment_message)
                        comment_id_list.append(comment_id)
                        reply_comment_message_list.append(reply_comment_message)
                        reply_created_time_list.append(reply_created_time)
                        reply_id_list.append(reply_id)
        except:
            pass
    
    # Create a dataframe from the lists
    df = pd.DataFrame({
        'post_id': post_id_list,
        'parent_comment_message': parent_comment_message_list,
        'comment_id': comment_id_list,
        'reply_comment_message': reply_comment_message_list,
        'reply_created_time': reply_created_time_list,
        'reply_id': reply_id_list
    })
    return df

def merge_dataframes(comments_df, posts_df, replies_df):
    # merge comments and posts dataframes on post_id
    merged_df = pd.merge(comments_df, posts_df, on='post_id', how='outer')
    # merge replies dataframe on comment_id
    merged_df = pd.merge(merged_df, replies_df, left_on='message_x', right_on='parent_comment_message', how='left')
    # rename columns
    merged_df = merged_df.rename(columns={
        'post_id_x': 'post_id',
        'created_time_x': 'comment_created_time',
        'created_time_y': 'post_created_time',
        'message_x': 'comment_message',
        'message_y': 'post_message',
        'parent_name_x': 'parent_name',
        'reply_comment_message': 'reply_message',
        'reply_created_time': 'reply_created_time',
        'reply_id': 'reply_id'
    })
    # drop unnecessary columns
    merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
    # reorder columns
    merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                           'comment_id', 'reply_message', 'reply_created_time', 'reply_id']]
    return merged_df

iteration = 1.1

# Get Facebook group posts
posts_data_dict[iteration] = get_facebook_posts(group_id, access_token)
posts_df_dict[iteration] = process_response(posts_data_dict[iteration])
posts_df_dict[iteration]

,post_id,created_time,message,comments_count,like_count,love_count,wow_count,haha_count,sad_count,angry_count,permalink_url
0,2139238999669147_3436553783270989,2023-06-29T21:59:11+0000,How good is food? 😍\n\nThe conclusion of my tr...,0,10,4,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
1,2139238999669147_3433787853547582,2023-06-25T22:09:04+0000,How good is a holiday ☀️\n\nTime off from norm...,0,11,5,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
2,2139238999669147_3433340976925603,2023-06-25T05:50:18+0000,Greetings team from Airlie Beach 👋\n\nReminder...,3,12,2,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
3,2139238999669147_3432149000378134,2023-06-23T06:46:23+0000,How I stay bikini confident year round 👙\n\nFi...,3,17,5,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
4,2139238999669147_3430738810519153,2023-06-20T20:59:53+0000,"Order of workouts, period pains and metabolism",2,12,4,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
5,2139238999669147_3430120997247601,2023-06-19T19:44:36+0000,A successful person doesn’t quit…Hear me out. ...,1,8,1,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
6,2139238999669147_3429515513974816,2023-06-18T22:13:44+0000,@everyone Live training this week will be on W...,3,1,1,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
7,2139238999669147_3429489833977384,2023-06-18T20:54:34+0000,Why I no longer work full time on the gym floo...,0,10,0,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
8,2139238999669147_3429124850680549,2023-06-18T07:38:58+0000,For so many years I dreamed of having abs…\n\n...,1,22,7,0,0,0,0,https://www.facebook.com/groups/strongandsassy...
9,2139238999669147_3427402890852745,2023-06-15T19:48:15+0000,Ah but it’s just a glass of wine…. It’s jus...,1,5,1,0,0,0,0,https://www.facebook.com/groups/strongandsassy...


In [ ]:

comments_df_dict[iteration], comments_response_dict[iteration] = get_fb_comments(group_id, access_token)
comments_df_dict[iteration]

In [ ]:
replies_dict[iteration] = create_reply_dataframe(comments_response_dict[iteration]['data'])
merged = merge_dataframes(comments_df_dict[iteration], posts_df_dict[iteration], replies_dict[iteration])
merged

## 1.11 pull 30 posts and their comments

In [ ]:

import requests
import pandas as pd

def get_facebook_posts(group_id, access_token, limit=30):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        pandas.DataFrame: A DataFrame containing the posts and their associated data.
    """
    url = f"https://graph.facebook.com/{group_id}/feed"

    params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(10).summary(true),reactions.limit(500).summary(true).type(LIKE).as(like),reactions.limit(500).summary(true).type(LOVE).as(love),reactions.limit(500).summary(true).type(WOW).as(wow),reactions.limit(500).summary(true).type(HAHA).as(haha),reactions.limit(500).summary(true).type(SAD).as(sad),reactions.limit(500).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    print("Response Status Code:", status_code)
    data = response.json()
    return data

## I will use this as part of the final code for the task
def process_response(data):
    posts = []
    comments = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })

    df = pd.DataFrame(posts)
    # Reorder columns
    df = df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow_count", "haha_count", "sad_count", "angry_count", "permalink_url"]]
    
    return df

def get_fb_comments(group_id, access_token, limit=30):
    """
    Pulls comments from a Facebook group using the Facebook API and outputs the results into a pandas DataFrame.
    
    Parameters:
    group_id (str): The ID of the Facebook group to pull comments from.
    access_token (str): A Facebook access token with the necessary permissions to access the group.
    
    Returns:
    Tuple(pandas.DataFrame, dict): A tuple containing a DataFrame with each comment as a row along with its details and replies,
    and the full JSON response from the API.
    """
    # Set up the API endpoint and parameters
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "fields": "comments{created_time,message,from,comments{created_time,message,from}}",
        "access_token": access_token,
        "limit": limit
    }

    # Make the GET request
    response = requests.get(url, params=params)
    
    # Convert the response to a DataFrame
    comments = []
    for post in response.json()["data"]:
        post_comments = post.get("comments", {}).get("data", [])
        for comment in post_comments:
            comment_dict = {
                "post_id": post.get("id"),
                "created_time": comment.get("created_time"),
                "message": comment.get("message"),
                "author_id": comment.get("from", {}).get("id"),
                "author_name": comment.get("from", {}).get("name"),
                "parent_id": None,
                "parent_name": None,
            }
            parent_comment = comment.get("comments", {}).get("data", [])
            if parent_comment:
                parent_comment = parent_comment[0]
                comment_dict["parent_id"] = parent_comment.get("id")
                comment_dict["parent_name"] = parent_comment.get("from", {}).get("name")
            comments.append(comment_dict)
    df = pd.DataFrame(comments)
    return df, response.json()

def create_reply_dataframe(json_object):
    # Initialize empty lists to store data
    post_id_list = []
    parent_comment_message_list = []
    comment_id_list = []
    reply_comment_message_list = []
    reply_created_time_list = []
    reply_id_list = []
    
    # Loop through each post in the json object
    for post in json_object:
        # Get the post ID
        post_id = post['id']
        
        try: # SH 2023-03-22 10:50 Added this 
            # Loop through each comment in the post
            for comment in post['comments']['data']:
                # Get the comment ID and message
                comment_id = comment['id']
                parent_comment_message = comment['message']
                
                # Check if the comment has any replies
                # if 'comments' in comment:
                if 'comments' in comment:
                    # Loop through each reply to the comment
                    for reply in comment['comments']['data']:
                        # Get the reply ID, message, and creation time
                        reply_id = reply['id']
                        reply_comment_message = reply['message']
                        reply_created_time = reply['created_time']
                        
                        # Add the data to the lists
                        post_id_list.append(post_id)
                        parent_comment_message_list.append(parent_comment_message)
                        comment_id_list.append(comment_id)
                        reply_comment_message_list.append(reply_comment_message)
                        reply_created_time_list.append(reply_created_time)
                        reply_id_list.append(reply_id)
        except:
            pass
    
    # Create a dataframe from the lists
    df = pd.DataFrame({
        'post_id': post_id_list,
        'parent_comment_message': parent_comment_message_list,
        'comment_id': comment_id_list,
        'reply_comment_message': reply_comment_message_list,
        'reply_created_time': reply_created_time_list,
        'reply_id': reply_id_list
    })
    return df

def merge_dataframes(comments_df, posts_df, replies_df):
    # merge comments and posts dataframes on post_id
    merged_df = pd.merge(comments_df, posts_df, on='post_id', how='outer')
    # merge replies dataframe on comment_id
    merged_df = pd.merge(merged_df, replies_df, left_on='message_x', right_on='parent_comment_message', how='left')
    # rename columns
    merged_df = merged_df.rename(columns={
        'post_id_x': 'post_id',
        'created_time_x': 'comment_created_time',
        'created_time_y': 'post_created_time',
        'message_x': 'comment_message',
        'message_y': 'post_message',
        'parent_name_x': 'parent_name',
        'reply_comment_message': 'reply_message',
        'reply_created_time': 'reply_created_time',
        'reply_id': 'reply_id'
    })
    # drop unnecessary columns
    merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
    # reorder columns
    merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                           'comment_id', 'reply_message', 'reply_created_time', 'reply_id']]
    return merged_df

iteration = 1.11

# Get Facebook group posts
posts_data_dict[iteration] = get_facebook_posts(group_id, access_token)
posts_df_dict[iteration] = process_response(posts_data_dict[iteration])
comments_df_dict[iteration], comments_response_dict[iteration] = get_fb_comments(group_id, access_token)
replies_dict[iteration] = create_reply_dataframe(comments_response_dict[iteration]['data'])
merged = merge_dataframes(comments_df_dict[iteration], posts_df_dict[iteration], replies_dict[iteration])
merged

# Iteration 2: Only create dataframe with original post and direct comments (exclude replies to comments)

In [ ]:
import sys
import requests
import pandas as pd

def get_facebook_posts(group_id, access_token, limit=30):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        pandas.DataFrame: A DataFrame containing the posts and their associated data.
    """
    url = f"https://graph.facebook.com/{group_id}/feed"

    params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(10).summary(true),reactions.limit(500).summary(true).type(LIKE).as(like),reactions.limit(500).summary(true).type(LOVE).as(love),reactions.limit(500).summary(true).type(WOW).as(wow),reactions.limit(500).summary(true).type(HAHA).as(haha),reactions.limit(500).summary(true).type(SAD).as(sad),reactions.limit(500).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    print("Response Status Code:", status_code)
    data = response.json()
    return data

## I will use this as part of the final code for the task
def process_response(data):
    posts = []
    comments = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })

    df = pd.DataFrame(posts)
    # Reorder columns
    df = df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow_count", "haha_count", "sad_count", "angry_count", "permalink_url"]]
    
    return df

def get_fb_comments(group_id, access_token, limit=30):
    """
    Pulls comments from a Facebook group using the Facebook API and outputs the results into a pandas DataFrame.
    
    Parameters:
    group_id (str): The ID of the Facebook group to pull comments from.
    access_token (str): A Facebook access token with the necessary permissions to access the group.
    
    Returns:
    Tuple(pandas.DataFrame, dict): A tuple containing a DataFrame with each comment as a row along with its details and replies,
    and the full JSON response from the API.
    """
    # Set up the API endpoint and parameters
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "fields": "comments{created_time,message,from,comments{created_time,message,from}}",
        "access_token": access_token,
        "limit": limit
    }

    # Make the GET request
    response = requests.get(url, params=params)
    
    # Convert the response to a DataFrame
    comments = []
    for post in response.json()["data"]:
        post_comments = post.get("comments", {}).get("data", [])
        for comment in post_comments:
            comment_dict = {
                "post_id": post.get("id"),
                "created_time": comment.get("created_time"),
                "message": comment.get("message"),
                "author_id": comment.get("from", {}).get("id"),
                "author_name": comment.get("from", {}).get("name"),
                "parent_id": None,
                "parent_name": None,
            }
            parent_comment = comment.get("comments", {}).get("data", [])
            if parent_comment:
                parent_comment = parent_comment[0]
                comment_dict["parent_id"] = parent_comment.get("id")
                comment_dict["parent_name"] = parent_comment.get("from", {}).get("name")
            comments.append(comment_dict)
    df = pd.DataFrame(comments)
    return df, response.json()

def create_reply_dataframe(json_object, add_replies=False):
    # Initialize empty lists to store data
    post_id_list = []
    parent_comment_message_list = []
    comment_id_list = []
    reply_comment_message_list = []
    reply_created_time_list = []
    reply_id_list = []
    
    # Loop through each post in the json object
    for post in json_object:
        # Get the post ID
        post_id = post['id']
        
        try: # SH 2023-03-22 10:50 Added this 
            # Loop through each comment in the post
            for comment in post['comments']['data']:
                # Get the comment ID and message
                comment_id = comment['id']
                parent_comment_message = comment['message']
                
                # Check if the comment has any replies
                if add_replies == True:
                    # Loop through each reply to the comment
                    for reply in comment['comments']['data']:
                        if 'comments' in comment:
                            # Get the reply ID, message, and creation time
                            reply_id = reply['id']
                            reply_comment_message = reply['message']
                            reply_created_time = reply['created_time']
                            
                            # Add the data to the lists
                            post_id_list.append(post_id)
                            parent_comment_message_list.append(parent_comment_message)
                            comment_id_list.append(comment_id)
                            reply_comment_message_list.append(reply_comment_message)
                            reply_created_time_list.append(reply_created_time)
                            reply_id_list.append(reply_id)
                else:
                    # Add the data to the lists
                    post_id_list.append(post_id)
                    parent_comment_message_list.append(parent_comment_message)
                    comment_id_list.append(comment_id)
        except Exception  as error:
            # exc_type, exc_obj, tb = sys.exc_info()
            # f = tb.tb_frame
            # lineno = tb.tb_lineno
            # filename = f.f_code.co_filename
            # print("An error occurred on line", lineno, "in", filename, ":", error)
            pass
    
    # Create a dataframe from the lists
    if add_replies == True:
        df = pd.DataFrame({
            'post_id': post_id_list,
            'parent_comment_message': parent_comment_message_list,
            'comment_id': comment_id_list,
            'reply_comment_message': reply_comment_message_list,
            'reply_created_time': reply_created_time_list,
            'reply_id': reply_id_list
        })
    else:
        df = pd.DataFrame({
            'post_id': post_id_list,
            'parent_comment_message': parent_comment_message_list,
            'comment_id': comment_id_list,
        })
    return df

def merge_dataframes(comments_df, posts_df, replies_df, add_replies=False):
    # merge comments and posts dataframes on post_id
    merged_df = pd.merge(comments_df, posts_df, on='post_id', how='outer')
    # merge replies dataframe on comment_id
    merged_df = pd.merge(merged_df, replies_df, left_on='message_x', right_on='parent_comment_message', how='left')
    if add_replies == True:
        # rename columns
        merged_df = merged_df.rename(columns={
            'post_id_x': 'post_id',
            'created_time_x': 'comment_created_time',
            'created_time_y': 'post_created_time',
            'message_x': 'comment_message',
            'message_y': 'post_message',
            'parent_name_x': 'parent_name',
            'reply_comment_message': 'reply_message',
            'reply_created_time': 'reply_created_time',
            'reply_id': 'reply_id'
        })
        # drop unnecessary columns
        merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
        # reorder columns
        merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                            'comment_id', 'reply_message', 'reply_created_time', 'reply_id']]
    else:
        # rename columns
        merged_df = merged_df.rename(columns={
            'post_id_x': 'post_id',
            'created_time_x': 'comment_created_time',
            'created_time_y': 'post_created_time',
            'message_x': 'comment_message',
            'message_y': 'post_message',
            'parent_name_x': 'parent_name',
        })
        # drop unnecessary columns
        merged_df = merged_df.drop(columns=['parent_name', 'parent_id'])
        # reorder columns
        merged_df = merged_df[['post_id', 'post_message', 'post_created_time', 'comment_created_time', 'parent_comment_message',
                            'comment_id']]
    return merged_df

iteration = 1.11

# # Get Facebook group posts
# posts_data_dict[iteration] = get_facebook_posts(group_id, access_token)
# posts_df_dict[iteration] = process_response(posts_data_dict[iteration])
# comments_df_dict[iteration], comments_response_dict[iteration] = get_fb_comments(group_id, access_token)
replies_dict[iteration] = create_reply_dataframe(comments_response_dict[iteration]['data'], add_replies=0)
merged = merge_dataframes(comments_df_dict[iteration], posts_df_dict[iteration], replies_dict[iteration])
merged

In [ ]:
for message in merged['parent_comment_message'].unique():
    print(message)

# Save outputs

In [43]:

outputs = {
    'FB_group_posts_data': posts_data_dict[iteration],
    'FB_group_posts_df': posts_df_dict[iteration],
    'FB_group_comments_df': comments_df_dict[iteration],
    'FB_group_comments_response': comments_response_dict[iteration],
    'FB_group_replies_df': replies_dict[iteration],
    'FB_group_posts_and_comments': merged
}

for output in outputs:
    save_output(outputs[output], description=output, csv_path=None, pickle_path='./outputs')

File saved:  ./outputs/FB_group_posts_data_2023-06-30_1550.sav
	Time completed: 2023-06-30 15:50:29.607287
	Object saved as pickle
File saved:  ./outputs/FB_group_posts_df_2023-06-30_1550.sav
	Time completed: 2023-06-30 15:50:29.621605
	Object saved as pickle
File saved:  ./outputs/FB_group_comments_df_2023-06-30_1550.sav
	Time completed: 2023-06-30 15:50:29.624605
	Object saved as pickle
File saved:  ./outputs/FB_group_comments_response_2023-06-30_1550.sav
	Time completed: 2023-06-30 15:50:29.627610
	Object saved as pickle
File saved:  ./outputs/FB_group_replies_df_2023-06-30_1550.sav
	Time completed: 2023-06-30 15:50:29.628608
	Object saved as pickle
File saved:  ./outputs/FB_group_posts_and_comments_2023-06-30_1550.sav
	Time completed: 2023-06-30 15:50:29.631607
	Object saved as pickle


# *End of Page*